In [1]:
import requests, time, os, datetime
import glob,re
import zipfile
import pandas as pd

In [2]:
working_directory = "./data"
country = 'us' #842
# country = 'mx' #484


In [3]:
def periodfYearMonth(year, month):
    period = str(year) + '0' + str(month) if month < 10 else str(year) + str(month)
    return period
def check_html_text(text):
    if 'USAGE LIMIT' in text:
        print(" USAGE LIMIT -- " + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
        return True
    if (text == "<html><body><h1>502 Bad Gateway</h1>\nThe server returned an invalid or incomplete response.\n</body></html>\n" or text == "Too Many Requests.\n" or text == "{\"Message\":\"An error has occurred.\"}"):
        with open(working_directory + "/comtradeusimport/" + "serverError.csv", 'a', encoding="utf-8") as log:
            log.write(str(datetime.datetime.now()) + ","+ str(url) + "," + str(filename) + "\n")
        return True
    return False
def retrieve_data(year, month, replace = False):
    try:
        period = periodfYearMonth(year, month)
        # url = f"https://comtrade.un.org/api/get/bulk/C/M/{period}/842/HS?token=W0YHqyAzFmNGp02eQf4+BIQVkydVa8NYI8iWZKJj1BmDAX7wKz4yau8v1V4HYODsO2tiao8U4BLgNm9Dd41k6/yrXjv5U11MzfYRQs6aIXHvQWXFIwhucgVSp88vEkIk"
        # url = f"https://comtrade.un.org/api/get/bulk/C/A/{year}/842/HS?token=W0YHqyAzFmNGp02eQf4+BIQVkydVa8NYI8iWZKJj1BmDAX7wKz4yau8v1V4HYODsO2tiao8U4BLgNm9Dd41k6/yrXjv5U11MzfYRQs6aIXHvQWXFIwhucgVSp88vEkIk"
        url = f"https://comtrade.un.org/api/get/bulk/C/M/{period}/842/HS?token=W0YHqyAzFmNGp02eQf4+BIQVkydVa8NYI8iWZKJj1BmDAX7wKz4yau8v1V4HYODsO2tiao8U4BLgNm9Dd41k6/yrXjv5U11MzfYRQs6aIXHvQWXFIwhucgVSp88vEkIk"
        filename = f"{country}import{period}.zip"
        print(f"Retriving {filename} -- " + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
        if not os.path.exists(working_directory + "/" + filename) or replace:
            content = requests.get(url, timeout=16)
            ''' note that sometimes we only get error informations in the responses, and here are some really dumb quick fixes'''
            if check_html_text(content.text):
                retrieve_data(year, month)
            else:
                # write csv file;
                with open(working_directory + "/" + filename, 'wb') as outfile:
                    outfile.write(content.content)
                # z = zipfile.ZipFile(io.BytesIO(content.content))
                # z.extractall(working_directory)
                print(f"{filename} download finished -- " + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    except requests.RequestException as e:
        print(type(e).__name__ + " has occurred")
        with open(working_directory + "/comtradeusimport/" + "exp.csv", 'a', encoding="utf-8") as log:
            log.write(str(datetime.datetime.now()) + ","+ str(url) + "," + str(filename) + "\n")
        print("\n" + content.content.decode())
        retrieve_data(year, month)
        return
def append_data(country, replace = True):
    i=0
    filelist = glob.glob(f"./data/{country}import[0-9]*.csv")
    month = []
    for file in filelist:
        match = re.search(r"([0-9]{6})", file.replace("\\","/"))
        if match:
            if not os.path.exists(f"./comtrade{country}_month.csv") or (replace and i == 0):
                pd.read_csv(file.replace("\\","/"), low_memory = False).to_csv(f"comtrade{country}_month.csv", index = False)
            else:
                pd.read_csv(file.replace("\\","/"), low_memory = False).to_csv(f"comtrade{country}_month.csv", index = False, header = False, mode = 'a')
            i+=1
            print(file)
                

In [4]:
for year in range(2010,2022+1):
    for month in range(1,12+1):
        if year<2022 or (year==2022 and month<=10):
            retrieve_data(year, month)

Retriving usimport201001.zip -- 2023-01-24 16:17:59
usimport201001.zip download finished -- 2023-01-24 16:22:40
Retriving usimport201002.zip -- 2023-01-24 16:22:40
Retriving usimport201003.zip -- 2023-01-24 16:22:40
Retriving usimport201004.zip -- 2023-01-24 16:22:40
Retriving usimport201005.zip -- 2023-01-24 16:22:40
Retriving usimport201006.zip -- 2023-01-24 16:22:40
Retriving usimport201007.zip -- 2023-01-24 16:22:40
Retriving usimport201008.zip -- 2023-01-24 16:22:40
Retriving usimport201009.zip -- 2023-01-24 16:22:40
Retriving usimport201010.zip -- 2023-01-24 16:22:40
Retriving usimport201011.zip -- 2023-01-24 16:22:40
Retriving usimport201012.zip -- 2023-01-24 16:22:40
Retriving usimport201101.zip -- 2023-01-24 16:22:40
Retriving usimport201102.zip -- 2023-01-24 16:22:40
Retriving usimport201103.zip -- 2023-01-24 16:22:40
Retriving usimport201104.zip -- 2023-01-24 16:22:40
Retriving usimport201105.zip -- 2023-01-24 16:22:40
Retriving usimport201106.zip -- 2023-01-24 16:22:40
Retr

In [6]:
zipfilelist = glob.glob(f"./data/{country}import*.zip")
for file in zipfilelist:
    match = re.search(r".*/data/(.*).zip",file.replace("\\","/"))
    filename = match.group(1)
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(working_directory)

PermissionError: [Errno 13] Permission denied: './data/usimport201605.zip'

In [ ]:
csvfilelist = glob.glob("./data/*.csv")
for file in csvfilelist:
    match = re.search(r"type-C_r-842_ps-([0-9]{6})_freq.*", file.replace("\\","/"))
    if match:
        newname = f"{country}import" + match.group(1)
        if os.path.exists(working_directory + "/"+ newname + ".csv"):
            os.remove(working_directory + "/"+ newname + ".csv")
        os.rename(file, working_directory + "/"+ newname + ".csv")

In [ ]:
# csvfilelist = glob.glob("./data/mximport*.csv")
# for file in csvfilelist:
#     match = re.search(r".*ps-([0-9]{4})_freq.*", file.replace("\\","/"))
#     if match:
#         newname = "mximport" + match.group(1)
#         os.rename(file, working_directory + "/"+ newname + ".csv")

In [ ]:
append_data(country)

./data/mximport201402.csv
./data/mximport201210.csv
./data/mximport201204.csv
./data/mximport201012.csv
./data/mximport201006.csv
./data/mximport201007.csv
./data/mximport201205.csv
./data/mximport201211.csv
./data/mximport201403.csv
./data/mximport201601.csv
./data/mximport201603.csv
./data/mximport202108.csv
./data/mximport201401.csv
./data/mximport201207.csv
./data/mximport201005.csv
./data/mximport201011.csv
./data/mximport201010.csv
./data/mximport201004.csv
./data/mximport201212.csv
./data/mximport201206.csv
./data/mximport202109.csv
./data/mximport201602.csv
./data/mximport201809.csv
./data/mximport201612.csv
./data/mximport201606.csv
./data/mximport201410.csv
./data/mximport201404.csv
./data/mximport201202.csv
./data/mximport201001.csv
./data/mximport201203.csv
./data/mximport201405.csv
./data/mximport201411.csv
./data/mximport201607.csv
./data/mximport201808.csv
./data/mximport201605.csv
./data/mximport201611.csv
./data/mximport201407.csv
./data/mximport201201.csv
./data/mximp

In [25]:
month = pd.read_csv(f"comtrade{country}_month.csv")

d:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
month[month['Commodity Code'].str.len() == 6]

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
2,HS,2010,201001,January 2010,6,1,1,Imports,484,Mexico,...,Ukraine,NaN,321511,"Ink; for printing, black, whether or not conce...",0,NaN,NaN,1.0,38,0
3,HS,2010,201001,January 2010,6,1,1,Imports,484,Mexico,...,Canada,NaN,321511,"Ink; for printing, black, whether or not conce...",0,NaN,NaN,436.0,4429,0
5,HS,2010,201001,January 2010,6,1,1,Imports,484,Mexico,...,United Kingdom,NaN,321000,"Paints and varnishes; (including enamels, lacq...",0,NaN,NaN,2.0,3,0
6,HS,2010,201001,January 2010,6,1,1,Imports,484,Mexico,...,Switzerland,NaN,320820,Paints and varnishes; based on acrylic or viny...,0,NaN,NaN,3740.0,71620,0
7,HS,2010,201001,January 2010,6,1,1,Imports,484,Mexico,...,United Kingdom,NaN,320820,Paints and varnishes; based on acrylic or viny...,0,NaN,NaN,3225.0,93411,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12017340,HS,2021,202112,December 2021,6,1,2,Exports,484,Mexico,...,Brazil,NaN,853641,"Electrical apparatus; relays, (for a voltage n...",0,NaN,NaN,8325.0,247715,0
12017341,HS,2021,202112,December 2021,6,1,2,Exports,484,Mexico,...,Canada,NaN,853641,"Electrical apparatus; relays, (for a voltage n...",0,NaN,NaN,243.0,13947,0
12017342,HS,2021,202112,December 2021,6,1,2,Exports,484,Mexico,...,Canada,NaN,853620,Electrical apparatus; automatic circuit breake...,0,NaN,NaN,37386.0,1908562,0
12017343,HS,2021,202112,December 2021,6,1,2,Exports,484,Mexico,...,El Salvador,NaN,853620,Electrical apparatus; automatic circuit breake...,0,NaN,NaN,734.0,26380,0


In [ ]:
# import pandas as pd
# filelist = glob.glob("./data/mximport[0-9]*.csv")
# year = []
# for file in csvfilelist:
#     match = re.search(r"([0-9]{4})", file.replace("\\","/"))
#     if match:
#         year.append(pd.read_csv(file, low_memory = False))


In [ ]:
# year = pd.concat(year)
# year.to_csv("comtrade_year.csv", index = False)
# del year

MemoryError: Unable to allocate 1.22 GiB for an array with shape (2, 81546977) and data type int64